In [1]:
import requests
import json
from string import Template
import ast

# LLM Setup

## Ollama Zephyr

In [ ]:
def ollama_generate(input: str, model = "zephyr-beta"):
    url = "http://localhost:11434/api/generate"
    
    payload = json.dumps({
      "model": model,
      "prompt": input,
      "stream": False
    })
    headers = {
      'Content-Type': 'application/json'
    }
    
    response = requests.request("POST", url, headers=headers, data=payload)
    return json.loads(response.text)['response'].strip()

In [ ]:
chat_template = Template("""<|system|>
You are a friendly chatbot who always helpful, harmless and honest.</s>
<|user|>
$input</s>
<|assistant|>""")

prompt = chat_template.substitute(input="Hello!")
print(ollama_generate(prompt))

## OpenAI

In [2]:
from openai import OpenAI
import os

In [3]:
client = OpenAI()

def openai_generate(input: str, system_message = "You are an assistant who is always helpful, harmless and honest.", model = "gpt-3.5-turbo"):
    completion = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": input}
        ]
    )
    
    return completion.choices[0].message.content

# Tools

In [38]:
from langchain.utilities import GoogleSerperAPIWrapper

def web_search(input: str):
    search = GoogleSerperAPIWrapper()
    return search.run(input)

# TODO: implement web_crawler with browserless or something similar for in-depth searches

In [5]:
def meaning_of_life():
    return "42 is the answer to life, the universe, and everything."

In [6]:
# TODO: Use dedent for all the prompts
def calculator(input: str):
    prompt_template = Template("""You are a calculator, perform the below Task,

Task: ```$task```

Return only the output. Do not add any text before or after the output.""")
    prompt = prompt_template.substitute(task=input)
    result = openai_generate(prompt)
    return result

In [102]:
def default_tool(input: str):
    prompt_template = Template("""You are a helpful, harmless and honest assistant. Perform the below Task, use the information in `Memory` for reference.

Memory:
```
$memory
```

Task: ```$task```

Return only the output. Do not add any text before or after the output.""")
    prompt = prompt_template.substitute(task=input, memory=get_memory())
    result = openai_generate(prompt)
    return result

# System State

In [103]:
system_state = {
    "task": "",
    "tools": {
        "web_search": {
            "func": web_search,
            "description": "Useful to get information on an unknown topic by searching it online. Input must be a string.",
            # "return_direct": True
        },
        "meaning_of_life": {
            "func": meaning_of_life,
            "description": "Useful to get the meaning of life. No inputs needed for this tool.",
        },
        "calculator": {
            "func": calculator,
            "description": "Useful to perform basic arithmetic operations. Input must be a string.",
        },
        "default": {
            "func": default_tool,
            "description": "This is a general purpose tool, which is good at most tasks. Input must be a string, it must contain the task to perform.",
        },
    },
    "current_plan": [],
    "short_term_memory": [],
}

# Utility Functions

In [77]:
def get_objective():
    return system_state['task']

In [78]:
def get_current_tools():
    tool_str = ""
    for idx, tool in system_state['tools'].items():
        # print(idx, tool['description'])
        tool_str += f"Name: {idx}\nDescription: {tool['description']}\n\n"
    return tool_str.strip()

In [79]:
def get_current_step():
    if len(system_state['current_plan']) == 0:
        return []
    return system_state['current_plan'][0]

In [80]:
def get_current_plan():
    plan = "[\n"
    for s in system_state['current_plan']:
        plan += "    " + s + ",\n"
    plan += "]"
    return plan

def set_current_plan(plan: str):
    system_state['current_plan'] = ast.literal_eval(plan)

def pretty_print_current_plan():
        print(f"""Current Plan:
```
{get_current_plan()}
```""")

In [137]:
def append_to_memory(current_step: str, action_output: str):
    system_state['short_term_memory'].append({'step': current_step, 'output': action_output})

def get_memory(limit: int = 3):
    memory = ""
    size = len(system_state['short_term_memory'])
    if size < limit:
        limit = size

    for s in system_state['short_term_memory'][size-limit:]:
        memory += "Step: " + s['step'] + "\n" + "Result: " + s['output'] + "\n\n"
    return memory.strip()

# Re-planner Agent

**Basic idea**: Based on Plan-and-execute type agents, but capable of more complex tasks as it is able to think and update the plan as and when needed. This helps it to tackle tasks with longer sequences.

**Flow**: Prepare a plan based on the task. Loop over the below step to proceed,
1. `Think`: Keeping the current step in mind, choose the right tool from the arsenel with right input.
2. `Act`: Execute the selected tool with the input from the previous step.
3. `Observe`: Based on the output of the action step, extract relevant information and update the current plan.

## Steps

### Plan

In [36]:
def prepare_plan(objective: str):
    planner_template = Template("""Your task is to analyze complex tasks and break them down into smaller unit sub-tasks.

Instructions:
* Do not say your knowledge is out of date, just return the requested information.
* Do not say you are a AI language model.
* Do not perform the task just yet, analyze and break down the task and return the sub-tasks.
* If a tool is used, sub-task must say something like, 'Use tool_a with X to do Y'.
* Use only the tools mentioned below if and when needed.
* Return the output in the form of an array, follow the below output format strictly.

Tools:
```
$tools
```

Output format:
```
[
    'sub-task 1', 
    'sub-task 2', 
    ...
]
```

Task: $input

Return the output in the specified format, do not deviate. Do not add any text before or after the output.
    """)
    
    prompt = planner_template.substitute(input=objective, tools=get_current_tools())
    # print(prompt)
    return openai_generate(prompt)

### Think

In [75]:
def think():
    thought_template = Template("""Your task is to extract the tool name and the input to the tool for the current step in the specified format.

Instructions:
* Do not say your knowledge is out of date, just return the requested information.
* Do not say you are a AI language model.
* Understand the `Current step` and extract the tool name and the complete input to the tool.
* Use the information from `Memory` if needed.
* Refer to the list of tools available to get the right tool name.
* Follow the below output format strictly.

Current step: `$current_step`

Tools:
```
$tools
```

Memory:
```
$memory
```

Output format:
```
{
    'current_step': "current step that is being analyzed",
    'tool_name': "name of the tool",
    'tool_input': "input to the tool with all the relevant information extracted",
}
```

Return the output in the specified format, do not deviate. Do not add any text before or after the output.
""")
    
    prompt = thought_template.substitute(current_step=get_current_step(), tools=get_current_tools(), memory=get_memory())
    # print(prompt)
    action_inputs = openai_generate(prompt)
    action = ast.literal_eval(action_inputs)
    return action['tool_name'], action['tool_input']

### Act

In [73]:
def execute_action(tool_name: str, tool_input):
    return system_state['tools'][tool_name]['func'](tool_input)

### Observe and Re-plan

In [138]:
def refresh_plan(action_output: str):
    observation_template = Template("""Your task is to observe the `Action Output` for the `Current Step` and update the `Current Plan` modifying it as needed. Return the updated plan as output in the specified format.

Instructions:
* Do not say your knowledge is out of date, just return the requested information.
* Do not say you are a AI language model.
* Update the plan by removing already executed steps or steps which are not necessary now.
* Less number of steps are preferred.
* Update any values in the next sub-tasks if they are available.
* Return `END_PLAN` as the only sub-task if all the steps are executed.
* You cannot have steps like, `note down x` or `write down y` or `save data to a variable`. All the data is already being saved to `Memory`.
* Follow the below output format strictly.

Current Step: `$current_step`

Action Output:
```
$action_output
```

Current Plan:
```
$current_plan
```

Memory:
```
$memory
```

Output format:
```
[
    'sub-task 1', 
    'sub-task 2', 
    ...
]
```

The output must be an array and every element must be in single quotes with a comma at the end, do not deviate. Do not add any text before or after the output.
""")
    
    prompt = observation_template.substitute(current_step=get_current_step(), action_output=action_output, current_plan=get_current_plan(), memory=get_memory())
    # print(prompt)
    refreshed_plan = openai_generate(prompt)
    return refreshed_plan

## Analyze data from memory and return the output

In [34]:
def analyze_memory():
    analyze_result_template = Template("""Based on each step executed and their result in the `Contextual Memory` and the `Task` at hand, analyze and return the answer.

Contextual Memory:
```
$memory
```

Task:
```
$task
```

Return only the output, do not add any text before or after the output.""")
    prompt = analyze_result_template.substitute(memory=get_memory(), task=get_objective())
    # print(prompt)
    result = openai_generate(prompt)
    return result

## Controller

In [139]:
objective = "Research about the current state of OpenAI. Get an outline, search for more information if needed and colate everything together into a coherent essay."
# objective = "Calculate the double of the age of Tom Cruise"

# Initialize system state
system_state['task'] = objective
system_state['short_term_memory'] = []
set_current_plan(prepare_plan(objective))
    
pretty_print_current_plan()

while "END_PLAN" not in str(system_state['current_plan'][0]):
    current_step = get_current_step()
    print(f"##### Execution Started #####")
    print(f"Executing Step: {get_current_step()}")

    # think
    print("\nTHINK: Keeping the current step in mind, choose the right tool from the list with the right input")
    tool_name, tool_input = think()
    print(f'Tool: {tool_name}, Tool inputs: {tool_input}')

    # act
    print("\nACT: Execute the selected tool with the input from the previous step")
    action_output = execute_action(tool_name, tool_input)
    print(f"Output of the current action: {action_output}")
    append_to_memory(current_step, action_output)

    # observe and refresh plan
    print("\nOBSERVE: Based on the output of the action step, extract relevant information and refresh the current plan")
    refreshed_plan = refresh_plan(action_output)
    set_current_plan(refreshed_plan)
    pretty_print_current_plan()

    print(f"##### Execution Done #####\n\n")

print("Analyzing memory to return the final result...")
result = analyze_memory()
print(f"FINAL RESULT: {result}")

Current Plan:
```
[
    Use web_search with "OpenAI current state" to gather information,
    Read multiple sources to get a comprehensive understanding of OpenAI,
    Take notes on key points and important details,
    Organize the information into an outline,
    Write an introduction for the essay,
    Write body paragraphs based on the outline, covering different aspects of OpenAI,
    Include information on recent developments, projects, and achievements,
    Include information on partnerships, collaborations, and funding,
    Include information on the impact of OpenAI in various fields,
    Write a conclusion summarizing the current state of OpenAI,
    Proofread and edit the essay for clarity and coherence,
]
```
##### Execution Started #####
Executing Step: Use web_search with "OpenAI current state" to gather information

THINK: Keeping the current step in mind, choose the right tool from the list with the right input
Tool: web_search, Tool inputs: OpenAI current state

ACT: 

# Archive

## Zephyr templates

In [ ]:
planner_template = Template("""<|system|>
You are an assistant who is always helpful, harmless and honest.</s>
<|user|>
Your task is to analyze complex tasks and break them down into smaller sub-tasks.

Instructions:
* Do not say your knowledge is out of date, just return the requested information.
* Do not say you are a AI language model.
* Do not perform the task just yet, analyze and break down the task and return the sub-tasks.
* If a tool is used, sub-task must say something like, 'Use tool_a with X to do Y'.
* Use only the tools mentioned below if and when needed.
* The sub-tasks must not say 'store the data'. All the data is by default stored in memory.
* Return the output in the form of an array, follow the below output format strictly.

Tools:
```
$tools
```

Output format:
```
[
    'sub-task 1', 
    'sub-task 2', 
    ...
]
```

Task: $input

Return the output in the specified format, do not deviate. Do not add any text before or after the output.</s>
<|assistant|>""")

prompt = planner_template.substitute(input="Calculate the double of the age of Tom Cruise", tools=get_current_tools())
# print(prompt)
set_current_plan(ollama_generate(prompt))

In [ ]:
thought_template = Template("""<|system|>
You are an assistant who is always helpful, harmless and honest.</s>
<|user|>
Your task is to extract the tool name and the input to the tool for the current step in the specified format.

Instructions:
* Do not say your knowledge is out of date, just return the requested information.
* Do not say you are a AI language model.
* Understand the `Current step` and extract the tool name and the complete input to the tool.
* Refer to the list of tools available to get the right tool name.
* Follow the below output format strictly.

Current step: `$current_step`

Tools:
```
$tools
```

Output format:
```
{
    'current_step': "current step that is being analyzed",
    'tool_name': "name of the tool",
    'tool_input': ["input 1", "input 2", ...]
}
```

Return the output in the specified format, do not deviate. Do not add any text before or after the output.</s>
<|assistant|>""")

prompt = action_template.substitute(current_step=get_current_step(), tools=get_current_tools())
# print(prompt)
print(ollama_generate(prompt))

## Manual Serper API Call

In [ ]:
url = "https://google.serper.dev/search"

payload = json.dumps({
  "q": "apple inc"
})
headers = {
  'X-API-KEY': '',
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

raw_snippets = json.loads(response.text)['organic']
for s in raw_snippets:
    print(s['snippet'])